In [1]:
from geevo.nodes import *
from geevo.graph import Graph2, Graph3

In [5]:
cond = "a > b"

In [8]:
a = 3
b = 2
s = cond.replace("a", str(a))
s = s.replace("b", str(b))
eval(s)

True

In [3]:
conf = {
    Source: 1,
    Pool: 2,
    Converter: 1,
    # Result: 1
}

In [19]:
Graph2.load("code/graphs/adc.pkl")

ModuleNotFoundError: No module named 'graph'

In [30]:
g = Graph2(config=conf, edge_list=[(0, 1), (1, 3), (3, 2)])

In [5]:
s_in = StateConnectionPoolRegister("a", output_pool_id=1, register_input_id=0)
s_out = StateConnectionRegisterEdge(output_register_id=0, edge_input_id=(3,2), modifier=10)
r = Register(condition="a > 2")
r.add_input(s_in)
r.add_output(s_out)

g3 = Graph3(config=conf, edge_list=[(0, 1), (1, 3), (3, 2)], weights=[[1, 1, 1, 1], []], registers=r)

IndexError: list index out of range